In [ ]:
from datetime import timedelta, datetime
import glob
import json
import os
import re

from collections import Counter

import numpy as np
import pandas as pd

import scipy.sparse as spr
import pickle

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [ ]:
val = pd.read_json('/val.json', typ = 'frame',encoding='utf-8')
train = pd.read_json('/train.json', typ = 'frame', encoding='utf-8')
test = pd.read_json('/test.json', typ = 'frame', encoding='utf-8')
genre_gn_all = pd.read_json('/genre_gn_all.json', typ = 'series',encoding='utf-8')

In [ ]:
song = pd.read_json('/song_meta.json', typ = 'frame',encoding='utf-8')

In [ ]:
plylst_tag = train['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])
tag_dict = {x: tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
    tag_id_tid[t] = i
    tag_tid_id[i] = t
n_tags = len(tag_dict)

plylst_song = train['songs']
song_dict = {x: x for x in song['id']}

n_songs = len(song_dict)

In [ ]:
train['tags_id'] = train['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

In [ ]:
# song genre 내용 가져오기. 
song_cate = []

for i in range(len(train)):
    gnr = [] 
    songs = train.iloc[i,3]
    
    for j in songs:
        for k in song.iloc[j,0]:
            gnr.append(k)
    song_cate.append(gnr)
    
    
train['plylst_genre'] = song_cate

In [ ]:
plylst_genre = train['plylst_genre']
genre_counter = Counter([gen for genre in plylst_genre for gen in genre])
genre_dict = {x: genre_counter[x] for x in genre_counter}

genre_id_tid = dict()
genre_tid_id = dict()
for i, t in enumerate(genre_dict):
    genre_id_tid[t] = i
    genre_tid_id[i] = t
n_genre = len(genre_dict)
train['plylst_genre_id'] = train['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])

In [ ]:
gnr_array = np.zeros((len(train),n_genre))
for i,index in enumerate(train.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(train.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_array[i][k] = c
gnr_array.shape

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


(115071, 219)

In [ ]:
train['num_songs'].sum()

5285871

In [ ]:
plylst_use = train[['plylst_title','updt_date','tags_id','songs']]
plylst_use.loc[:,'num_songs'] = plylst_use['songs'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
plylst_train = plylst_use
print("train data의 shape는 : {}".format(plylst_train.shape))

train data의 shape는 : (115071, 6)


# Spare Matrix Making

### User- Song / User-Tag / User - Genre / User - Date 

In [ ]:
!pip install sentencepiece

In [ ]:
from sentencepiece import SentencePieceProcessor
from sklearn.feature_extraction.text import CountVectorizer

sp = SentencePieceProcessor()
sp.Load("/playlist_title.model")

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000, tokenizer=sp.encode_as_pieces)
# content = train['plylst_title'] + val['plylst_title'] + test['plylst_title'] # train/val/test 추가
content = train['plylst_title']
tdm = cv.fit_transform(content)

title_tdm = tdm.toarray()

In [ ]:
n_train = len(plylst_train)
row = np.repeat(range(n_train), plylst_train['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in plylst_train['songs'] for song in songs] # Song dic number 추출 
dat = np.repeat(1, plylst_train['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
train_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs)) # csr_matrix 제작 

row = np.repeat(range(n_train), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_train['num_tags'].sum())
train_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))

In [ ]:
len([song for songs in plylst_train['songs'] for song in songs])

5285871

In [ ]:
train_user_songs_A_T = train_user_songs_A.T.tocsr() 
train_user_songs_A_T # 행에는 노래 columns에는 User 정보 삽입 

<707989x115071 sparse matrix of type '<class 'numpy.longlong'>'
	with 5285871 stored elements in Compressed Sparse Row format>

In [ ]:
train_user_tags_A_T = train_user_tags_A.T.tocsr() 
train_user_tags_A_T # 행에는 Tangs columns에는 User 정보 삽입 

<29160x115071 sparse matrix of type '<class 'numpy.longlong'>'
	with 476331 stored elements in Compressed Sparse Row format>

In [ ]:
# import tensorflow as tf

# def like_cnt_def(x):
#     if x <1:
#         return 0
#     elif x<10:
#         return 1
#     elif x<50:
#         return 2
#     elif x<100:
#         return 3
#     elif x<1000:
#         return 4
#     else:
#         return 5
# like_ = train['like_cnt'].map(like_cnt_def)
# like_cnt = tf.keras.utils.to_categorical(like_, 6)

## test Model Define

In [ ]:
song_cate = []

for i in range(len(val)):
    gnr = [] 
    songs = val.iloc[i,3]
    
    for j in songs:
        for k in song.iloc[j,0]:
            gnr.append(k)
    song_cate.append(gnr)
    
val['plylst_genre'] = song_cate

In [ ]:
val['tags_id'] = val['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])
val['plylst_genre_id'] = val['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])
val.loc[:,'num_songs'] = val['songs'].map(len)
val.loc[:,'num_tags'] = val['tags_id'].map(len)
val_title = cv.transform(val['plylst_title']).toarray()

In [ ]:
gnr_val = np.zeros((len(val),n_genre))
for i,index in enumerate(val.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(val.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_val[i][k] = c
gnr_val.shape

0
10000
20000


(23015, 219)

In [ ]:
n_val = len(val)
row = np.repeat(range(n_val), val['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in val['songs'] for song in songs] # Song dic number 추출 
dat = np.repeat(1, val['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
val_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_val, n_songs)) # csr_matrix 제작 

row = np.repeat(range(n_val), val['num_tags'])
col = [tag for tags in val['tags_id'] for tag in tags]
dat = np.repeat(1, val['num_tags'].sum())
val_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_val, n_tags))

In [ ]:
val_user_songs_A_T = val_user_songs_A.T.tocsr()
val_user_tags_A_T = val_user_tags_A.T.tocsr()

# test

In [ ]:
song_cate = []

for i in range(len(test)):
    gnr = [] 
    songs = test.iloc[i,3]
    
    for j in songs:
        for k in song.iloc[j,0]:
            gnr.append(k)
    song_cate.append(gnr)
    
test['plylst_genre'] = song_cate

In [ ]:
test['tags_id'] = test['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])
test['plylst_genre_id'] = test['plylst_genre'].map(lambda x: [genre_id_tid.get(s) for s in x if genre_id_tid.get(s) != None])
test.loc[:,'num_songs'] = test['songs'].map(len)
test.loc[:,'num_tags'] = test['tags_id'].map(len)
test_title = cv.transform(test['plylst_title']).toarray()

gnr_test = np.zeros((len(test),n_genre))
for i,index in enumerate(test.index):
    if i%10000 == 0:
        print(i)
    counter = Counter(test.loc[index]['plylst_genre_id'])
    for (k,c) in counter.items():
        gnr_test[i][k] = c
gnr_test.shape

0
10000


(10740, 219)

In [ ]:
n_test = len(test)
row = np.repeat(range(n_test), test['num_songs']) # User Index 별 노래 개수만큼 만듦 
col = [song for songs in test['songs'] for song in songs] # Song dic number 추출 
dat = np.repeat(1, test['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
test_user_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_songs)) # csr_matrix 제작 

row = np.repeat(range(n_test), test['num_tags'])
col = [tag for tags in test['tags_id'] for tag in tags]
dat = np.repeat(1, test['num_tags'].sum())
test_user_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_tags))

test_user_songs_A_T = test_user_songs_A.T.tocsr()
test_user_tags_A_T = test_user_tags_A.T.tocsr()

### Model 설명
* CF를 사용하였음

* 이때 valid data에 song 정보가 존재하는 경우 song 활용 유사행렬 구해서 작업 진행 

* song 정보가 없고 tag 정보만 존재할 때 tag 정보 활용 유사한 유저 추출 

* 둘다 없는 경우 Title 활용하여 유사한 유저 추출 

In [ ]:
title_gnr = np.concatenate((gnr_array,title_tdm),axis=1)
val_title_gnr = np.concatenate((gnr_val,val_title),axis=1)
test_title_gnr = np.concatenate((gnr_test,test_title),axis=1)

In [ ]:
from scipy.sparse import vstack 
from scipy import sparse

title_sp = sparse.csr_matrix(title_tdm)

In [ ]:
title_gnr = sparse.csr_matrix(title_gnr)
val_title_gnr = sparse.csr_matrix(val_title_gnr)
test_title_gnr = sparse.csr_matrix(test_title_gnr)

In [ ]:
title_gnr = vstack([title_gnr,val_title_gnr,test_title_gnr])
song_sp = vstack([train_user_songs_A,val_user_songs_A,test_user_songs_A])
tag_sp = vstack([train_user_tags_A,val_user_tags_A,test_user_tags_A])

In [ ]:
song_sp_T = song_sp.T.tocsr()
tag_sp_T = tag_sp.T.tocsr()

In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn_song = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)
model_knn_tag = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)
model_knn_title = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)
model_knn_title_gnr = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=25, n_jobs=-1)

In [ ]:
model_knn_song.fit(song_sp)
model_knn_tag.fit(tag_sp)
model_knn_title.fit(title_sp)
model_knn_title_gnr.fit(title_gnr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=25, p=2,
                 radius=1.0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
train.loc[:,'num_songs'] = train['songs'].map(len)
train.loc[:,'num_tags'] = train['tags_id'].map(len)

In [ ]:
data_all = pd.concat([train,val,test])

In [ ]:
data_all.index = range(len(data_all))

In [ ]:
from tqdm import tqdm_notebook
res = []
for i in tqdm_notebook(range(len(val))):
    data = val.iloc[i]
    pid = i
    
    if len(data['songs']) != 0 and len(data['tags_id']) != 0:
        p = np.zeros((707989,1))
        p[data['songs']] = 1

        pp = np.zeros((n_tags,1))
        pp[data['tags_id']] = 1

        tra_song = data_all.iloc[model_knn_song.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(25), tra_song['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_song['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_song['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        tra_tag = data_all.iloc[model_knn_tag.kneighbors(pp.T)[1][0]]
        row = np.repeat(range(25), tra_tag['num_tags'])
        col = [tag for tags in tra_tag['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_tag['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        tra_title_gnr = title_gnr[model_knn_title_gnr.kneighbors(val_title_gnr[i:(i+1)])[1][0]]
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]
        
        val_song = cosine_similarity(tra_song_sp,p.T)
        val_tag = cosine_similarity(tra_tag_sp,pp.T)
        val_title_genre = cosine_similarity(tra_title_gnr,val_title_gnr[i:(i+1)])
        
        m = 2*(val_song.max())
        vali = (val_song/m) * val_tag * val_title_genre
        
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-300:][::-1] # 값이 높은 상위 150개 노래 추출 

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]  # 중복되는 노래 있는지 확인하고 100개 추출 

        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]
        
        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                        })
    
    # song만 있을 때
    elif len(data['songs']) != 0:
        p = np.zeros((707989,1))
        p[data['songs']] = 1
        
        tra_song = data_all.iloc[model_knn_song.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(25), tra_song['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_song['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_song['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        row = np.repeat(range(25), tra_song['num_tags'])
        col = [tag for tags in tra_song['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_song['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]
        
        tra_title_gnr = title_gnr[model_knn_title_gnr.kneighbors(val_title_gnr[i:(i+1)])[1][0]]
        
        val_song = cosine_similarity(tra_song_sp,p.T)
        val_title_genre = cosine_similarity(tra_title_gnr,val_title_gnr[i:(i+1)])

        vali = val_song*val_title_genre
        
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-200:][::-1] # 값이 높은 상위 150개 노래 추출 

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]  # 중복되는 노래 있는지 확인하고 100개 추출 

        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]
        
        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })
        
    # tag만 있을 때    
    elif len(data['tags_id']) !=0:
        p = np.zeros((n_tags,1))
        p[data['tags_id']] = 1
        
        tra_tag = data_all.iloc[model_knn_tag.kneighbors(p.T)[1][0]]   
        row = np.repeat(range(25), tra_tag['num_songs']) # User Index 별 노래 개수만큼 만듦 
        col = [song for songs in tra_tag['songs'] for song in songs] # Song dic number 추출 
        dat = np.repeat(1, tra_tag['num_songs'].sum()) # User별 Song이 있는 부분에 1을 넣기위해 1과 전체 노래 개수만큼 만듦
        tra_song_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_songs)) # csr_matrix 제작 
        tra_song_sp_T = tra_song_sp.T.tocsr()
        
        row = np.repeat(range(25), tra_tag['num_tags'])
        col = [tag for tags in tra_tag['tags_id'] for tag in tags]
        dat = np.repeat(1, tra_tag['num_tags'].sum())
        tra_tag_sp = spr.csr_matrix((dat, (row, col)), shape=(25, n_tags))
        tra_tag_sp_T = tra_tag_sp.T.tocsr()
        
        
        songs_already = data["songs"] 
        tags_already = data["tags_id"]
        
        vali = cosine_similarity(tra_tag_sp,pp.T)
        
        if len(data['plylst_title']) != 0 :
            tra_title_gnr = title_tdm[model_knn_title.kneighbors(val_title[i:(i+1)])[1][0]]
            vali_title = cosine_similarity(tra_title_gnr,val_title[i:(i+1)])
            vali = vali * vali_title
                
        cand_song = tra_song_sp_T.dot(vali) # 행에는 노래 열에는 유저 정보 %*% 유사한 유저 -> 유사한 노래에 대하여 높은 값 나옴 
        cand_song_idx = cand_song.reshape(-1).argsort()[-300:][::-1] # 값이 높은 상위 150개 노래 추출 

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]  # 중복되는 노래 있는지 확인하고 100개 추출 

        cand_tag = tra_tag_sp_T.dot(vali) # 똑같은 작업 실시 
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-30:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]

        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })
        
    else:
        cand_song = []
        for li in train.iloc[model_knn_title.kneighbors(val_title[i:(i+1)])[1][0]].songs.to_list():
            for j in li:
                cand_song.append(j)
                
        cand_tag = []
        for li in train.iloc[model_knn_title.kneighbors(val_title[i:(i+1)])[1][0]].tags.to_list():
            for j in li:
                cand_tag.append(j)
        
        cand_song_idx = list(pd.DataFrame(cand_song)[0].value_counts()[:100].index)
        rec_tag_idx = list(pd.DataFrame(cand_tag)[0].value_counts()[:10].index)

        res.append({
                    "id": val.loc[pid,'id'],
                    "songs": cand_song_idx,
                    "tags": rec_tag_idx
                    })

for i in range(len(res)):
    if len(res[i]['songs']) != 100:
        print('song 에서 {}번째 오류 발생'.format(i))
        
    if len(res[i]['tags']) != 10:
        print('tag 에서 {}번째 오류 발생'.format(i))

import io
import os

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

rec = []
for i in range(len(res)):
    rec.append({
                    "id": res[i]['id'],
                    "songs": list(res[i]['songs']),
                    "tags": res[i]['tags']
                    })
    
write_json(rec, "/content/gdrive/My Drive/Colab Notebooks/머신/2323_results.json")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: ignored

In [ ]:
write_json(rec, "/content/gdrive/My Drive/Colab Notebooks/머신/it_its_results.json")

In [ ]:
hyn = pd.read_json('/content/gdrive/My Drive/Colab Notebooks/머신/it_its_results.json', typ = 'frame',encoding='UTF-8')